In [13]:
from ase import Atoms
from ase.build import add_adsorbate, fcc100
from mace.calculators import mace_mp
from ase.build import surface
from ase.constraints import FixAtoms, UnitCellFilter
from ase.optimize import QuasiNewton
from ase.visualize import view
from ase.io import read, write
from ase.optimize import LBFGS
import numpy as np
from ase.build.supercells import make_supercell
from cu2o_bulk import cu2o_bulk, cu2o100, Oterm1x1, c2x2

bulk = read('/home/lana/Downloads/9007497.cif')
#view(bulk)

bulk = cu2o_bulk()

bulk.calc = mace_mp(model="small", dispersion=False, default_dtype="float64", device='cpu')
ucf = UnitCellFilter(bulk)
LBFGS(ucf).run(fmax=0.01)

E_bulk = bulk.get_potential_energy()

n_layers=4
slab = Oterm1x1(bulk, 4, 10)
slab=make_supercell(slab, [[3,0,0], [1,1, 0],  [0,0,1]] )
Max_Cu_z = np.max(slab[slab.symbols=='Cu'].positions[:,2]) - 2.0
mask2=(slab.positions[:, 2] >= Max_Cu_z) & (slab.symbols=='Cu')
del slab[mask2]
Max_O_z = np.max(slab[slab.symbols=='Cu'].positions[:,2]) - 0.5
mask3=(slab.positions[:, 2] >= Max_O_z) & (slab.symbols=='O')
del slab[mask3]
#del  slab[13] 
#del slab[51]
#slab.calc = mace_mp(model="small", dispersion=False, default_dtype="float64", device='cpu')
#view(slab)

'''
E_slab = slab.get_potential_energy()
E_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
print(f'{n_layers=} {E_cleav=}')
'''

bottom_Cu_z = np.min(slab[slab.symbols=='Cu'].positions[:,2])
mask1=slab.positions[:, 2] < bottom_Cu_z + 1.0
slab.set_constraint(FixAtoms(mask=mask1))
slab.calc = mace_mp(model="small", dispersion=False, default_dtype="float64", device='cpu')
view(slab)

'''

qn = LBFGS(slab, trajectory='111slab.traj')
qn.run(fmax=0.01)
#E_slab = slab.get_potential_energy()
t = read('111slab.traj@:')
view(t)


#E_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
#print(f'{n_layers=} {E_surf=}')
'''

Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/46jrkm3v
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 15:11:21      -28.924518*       0.0261
LBFGS:    1 15:11:22      -28.924546*       0.0243
LBFGS:    2 15:11:22      -28.924723*       0.0001
Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/46jrkm3v
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


"\n\nqn = LBFGS(slab, trajectory='111slab.traj')\nqn.run(fmax=0.01)\n#E_slab = slab.get_potential_energy()\nt = read('111slab.traj@:')\nview(t)\n\n\n#E_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])\n#print(f'{n_layers=} {E_surf=}')\n"

In [11]:
!pip install mace-torch ase


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
